# Importing Libraries and Modules

In [ ]:
import os
import glob

import pandas as pd
import numpy as np

import tensorflow as tf

from tqdm.notebook import tqdm 

# Path Initialization

This cell initializes the paths to the dataset and the models.

+ `PATH_DATASET` is the directory where the synthetic data is saved.
+ `PATH_SET` is the complete path to the test dataset file within the dataset directory.
+ `PATH_MODELS` is the directory where the model files are located.

In [ ]:
PATH_DATASET = '../../2_training/data/datasets/'
PATH_SET = f'{PATH_DATASET}test_dataset.npy'
PATH_MODELS = '../models/'

# Model Files Retrieval

This cell retrieves the list of all model files located in the PATH_MODELS directory. The glob function is used to get all files with the .h5 extension, representing the saved model files.

In [ ]:
models = glob.glob(f'{PATH_MODELS}*.keras')
models

# Dataset Loading

This cell loads the test dataset from the PATH_SET into the variable dataset. The dataset is expanded along a new axis to match the input shape required by the models.

In [ ]:
dataset = np.load(PATH_SET)[..., tf.newaxis]

# Model Loading and Prediction

In this cell, each `model` in the `models` list is loaded and used to predict the labels of the instances in the dataset. For each model, a new DataFrame `table_labels` is created to store the index (subint) and the predicted label (`models_label`) for each instance in the dataset. After predicting labels for all instances in the dataset with a model, the corresponding `table_labels` DataFrame is saved as a CSV file, with the filename indicating which model was used for prediction. The tqdm is used here to display progress bars for model loading and label prediction processes.

In [ ]:
for model in tqdm(models):
    loaded_model = tf.keras.models.load_model(model)
    table_labels = pd.DataFrame(columns=['subint', 'models_label'])
    for idx, image in enumerate(tqdm(dataset)):
        image = tf.expand_dims(image, 0)
        model_prediction = loaded_model.predict(image)
        label = np.argmax(model_prediction)
        table_labels.loc[idx] = [
            idx, 
            label
        ]
    
    table_labels.to_csv(f'../models_labels/labels_real_data_by_{os.path.basename(model).split(".")[0]}.csv')